# Checking out the data
Largely based on https://github.com/dfiorino/audl-pull/blob/master/examples/01_intro-guide.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from src.data.utils import load_raw_data, DATA_DIR

In [44]:
%%time
# Run data extraction
from src.data.process_team_indicators import make_team_indicators
make_team_indicators()

Wall time: 11.8 s


### Here is the final data we're going for

In [45]:
df = pd.read_csv('../data/processed/team_indicators.csv')

In [47]:
df.shape

(1700, 12)

In [46]:
df.head()

,year,team,opponent,date,game,Goals,Catches,Ds,Turnovers,Drops,Throwaways,Goals_against
0,2014,Chicago Wildfire,Cincinnati Revolution,2014-04-18,Chicago Wildfire_Cincinnati Revolution_2014-04-18,22,192.0,7.0,31.0,3.0,28.0,23
1,2014,Chicago Wildfire,Cincinnati Revolution,2014-04-26,Chicago Wildfire_Cincinnati Revolution_2014-04-26,27,185.0,23.0,54.0,8.0,46.0,12
2,2014,Chicago Wildfire,Cincinnati Revolution,2014-06-28,Chicago Wildfire_Cincinnati Revolution_2014-06-28,21,273.0,17.0,51.0,10.0,41.0,11
3,2014,Chicago Wildfire,Detroit Mechanix,2014-05-03,Chicago Wildfire_Detroit Mechanix_2014-05-03,18,193.0,21.0,78.0,15.0,63.0,9
4,2014,Chicago Wildfire,Detroit Mechanix,2014-06-04,Chicago Wildfire_Detroit Mechanix_2014-06-04,31,154.0,8.0,43.0,2.0,41.0,17


### Here is the process by which it happens in the repo

In [4]:
df = pd.read_csv(f'{DATA_DIR}/raw/AUDL2018_MadisonRadicals.csv', index_col=0)

In [5]:
df.head()

,Date/Time,Opponent,Point Elapsed Seconds,Line,Our Score - End of Point,Their Score - End of Point,Event Type,Action,Passer,Receiver,...,Begin Y,End Area,End X,End Y,Distance Unit of Measure,Absolute Distance,Lateral Distance,Toward Our Goal Distance,Teamname,Tournament
0,2018-04-07 19:38,Indianapolis AlleyCats,44,O,1,0,Offense,Catch,Ben Nelson,Kevin Brown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Madison Radicals,AUDL 2018
1,2018-04-07 19:38,Indianapolis AlleyCats,44,O,1,0,Offense,Catch,Kevin Brown,Ben Nelson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Madison Radicals,AUDL 2018
2,2018-04-07 19:38,Indianapolis AlleyCats,44,O,1,0,Offense,Catch,Ben Nelson,Tom Annen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Madison Radicals,AUDL 2018
3,2018-04-07 19:38,Indianapolis AlleyCats,44,O,1,0,Offense,Catch,Tom Annen,Pat Shriwise,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Madison Radicals,AUDL 2018
4,2018-04-07 19:38,Indianapolis AlleyCats,44,O,1,0,Offense,Catch,Pat Shriwise,Kevin Brown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Madison Radicals,AUDL 2018


In [6]:
df['Action'].value_counts()

Catch                3966
Goal                  722
Throwaway             421
Pull                  413
D                     238
Drop                   61
PullOb                 34
EndOfThirdQuarter      17
EndOfFirstQuarter      17
Halftime               17
GameOver               17
Stall                   7
Name: Action, dtype: int64

# Aggregate indicators

In [7]:
# Make index
df['datetime'] = pd.to_datetime(df['Date/Time'])
df['date'] = df.datetime.dt.date
df['year'] = df.datetime.dt.year
df['team'] = df['Teamname']
df['opponent'] = df['Opponent']
df['game'] = df.team + "_" + df.opponent + "_" + df.date.map(str)
index_vars = ['year', 'team', 'opponent', 'game']

In [8]:
# Make dummy variables, as this will help in aggregation
dummies = pd.get_dummies(df['Action'])
dummies.head()

,Catch,D,Drop,EndOfFirstQuarter,EndOfThirdQuarter,GameOver,Goal,Halftime,Pull,PullOb,Stall,Throwaway
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0


In [9]:
df = pd.concat([df, dummies], axis=1)

# NOTE - for goals, we want to separate whether the team is on Offense or Defense
df['Goals_against'] = 0
df.loc[df['Event Type'] == 'Defense', 'Goals_against'] = df['Goal']
df.loc[df['Event Type'] == 'Defense', 'Goal'] = 0
df['Turnovers'] = df.Throwaway + df.Drop

In [10]:
# Let's be intensional with indicator names 
rename_ind_dict = {
    'Goal':'Goals',
    'Catch':'Catches',
    'D': 'Ds',
    'Drop': 'Drops',
    'Throwaway': 'Throwaways',
}
df.rename(columns=rename_ind_dict, inplace=True)

indicators = ['Goals', 'Catches', 'Ds', 'Turnovers', 'Drops', 'Throwaways', 'Goals_against']


In [11]:
df_wide = df.groupby(index_vars)[indicators].sum()
df_wide.head()

Goals  \
year team             opponent          game                                                   
2018 Madison Radicals Chicago Wildfire  Madison Radicals_Chicago Wildfire_2018-05-19      24   
                                        Madison Radicals_Chicago Wildfire_2018-06-09      22   
                                        Madison Radicals_Chicago Wildfire_2018-07-14      27   
                      Dallas Roughnecks Madison Radicals_Dallas Roughnecks_2018-08-12     20   
                      Detroit Mechanix  Madison Radicals_Detroit Mechanix_2018-04-21      27   

                                                                                       Catches  \
year team             opponent          game                                                     
2018 Madison Radicals Chicago Wildfire  Madison Radicals_Chicago Wildfire_2018-05-19     205.0   
                                        Madison Radicals_Chicago Wildfire_2018-06-09     252.0   
                                        Madison Radicals_Chicago Wildfire_2018-07-14     313.0   
                      Dallas Roughnecks Madison Radicals_Dallas Roughnecks_2018-08-12    174.0   
                      Detroit Mechanix  Madison Radicals_Detroit Mechanix_2018-04-21     200.0   

                                                                                         Ds  \
year team             opponent          game                                                  
2018 Madison Radicals Chicago Wildfire  Madison Radicals_Chicago Wildfire_2018-05-19   13.0   
                                        Madison Radicals_Chicago Wildfire_2018-06-09    7.0   
                                        Madison Radicals_Chicago Wildfire_2018-07-14   12.0   
                      Dallas Roughnecks Madison Radicals_Dallas Roughnecks_2018-08-12  11.0   
                      Detroit Mechanix  Madison Radicals_Detroit Mechanix_2018-04-21   22.0   

                                                                                       Turnovers  \
year team             opponent          game                                                       
2018 Madison Radicals Chicago Wildfire  Madison Radicals_Chicago Wildfire_2018-05-19        29.0   
                                        Madison Radicals_Chicago Wildfire_2018-06-09        35.0   
                                        Madison Radicals_Chicago Wildfire_2018-07-14        22.0   
                      Dallas Roughnecks Madison Radicals_Dallas Roughnecks_2018-08-12       21.0   
                      Detroit Mechanix  Madison Radicals_Detroit Mechanix_2018-04-21        39.0   

                                                                                       Drops  \
year team             opponent          game                                                   
2018 Madison Radicals Chicago Wildfire  Madison Radicals_Chicago Wildfire_2018-05-19     0.0   
                                        Madison Radicals_Chicago Wildfire_2018-06-09     6.0   
                                        Madison Radicals_Chicago Wildfire_2018-07-14     2.0   
                      Dallas Roughnecks Madison Radicals_Dallas Roughnecks_2018-08-12    0.0   
                      Detroit Mechanix  Madison Radicals_Detroit Mechanix_2018-04-21     6.0   

                                                                                       Throwaways  \
year team             opponent          game                                                        
2018 Madison Radicals Chicago Wildfire  Madison Radicals_Chicago Wildfire_2018-05-19         29.0   
                                        Madison Radicals_Chicago Wildfire_2018-06-09         29.0   
                                        Madison Radicals_Chicago Wildfire_2018-07-14         20.0   
                      Dallas Roughnecks Madison Radicals_Dallas Roughnecks_2018-08-12        21.0   
                      Detroit Mechanix  Madison Radicals_Detroit Mechanix_2018-04